In [13]:
import sys  
sys.path.insert(0, './../')
import numpy as np
import bbdc2021 as bbdc
import pandas as pd
from importlib import reload
reload(bbdc)
import tensorflow as tf
from keras import layers
import keras
from keras import backend as K
from itertools import groupby

In [2]:
X_train, Y_train, timepoints, trainDataframe = bbdc.load_data("train.csv", "dataset_fft/")
X_validation, Y_validation, timepoints, validationDataframe = bbdc.load_data("validation.csv", "dataset_fft/")  
X_test, Y_test, timepoints, testDataframe = bbdc.load_data("test.csv", "dataset_fft/")

X_train_cropped = [np.delete(x,1,0) for x in X_train] #TODO besser lösen
Y_train_cropped = [np.delete(x,1,0) for x in Y_train] #TODO besser lösen
X_validation_cropped = [np.delete(x,1,0) for x in X_validation] #TODO besser lösen
Y_validation_cropped = [np.delete(x,1,0) for x in Y_validation] #TODO besser lösen
X_test_cropped = [np.delete(x,1,0) for x in X_test] #TODO besser lösen
Y_test_cropped = [np.delete(x,1,0) for x in Y_test] #TODO besser lösen
inputShape = X_train_cropped[0].shape

In [3]:
channels = [32, 64, 128]

#input_layer = layers.Input(shape=(296,296,1))
input_layer = layers.Input(shape=(inputShape))
##############################
x = input_layer
# Encoder Start
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh1 = layers.ReLU()(x)

x = layers.MaxPooling1D()(sh1)
#############################
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
sh2 = layers.ReLU()(x)
x = layers.MaxPooling1D()(sh2)
#############################
x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[2], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
#Decoder start
x = layers.UpSampling1D(2)(x)

############################
#sh2 = layers.Cropping1D(cropping=10)(sh2)
x = layers.Concatenate()([x,sh2])
x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[1], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)
x = layers.UpSampling1D(2)(x)
#############################
#sh1 = layers.Cropping1D(cropping=16)(sh1)
x = layers.Concatenate()([x,sh1])
x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(channels[0], kernel_size=3, padding='same')(x)
x = layers.BatchNormalization()(x)
x = layers.ReLU()(x)

x = layers.Conv1D(13, kernel_size=1, activation="softmax", padding='same')(x)

model = keras.models.Model(inputs=input_layer, outputs=x)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 208, 17)      0                                            
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 208, 32)      1664        input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 208, 32)      128         conv1d_1[0][0]                   
__________________________________________________________________________________________________
re_lu_1 (ReLU)                  (None, 208, 32)      0           batch_normalization_1[0][0]      
____________________________________________________________________________________________

In [32]:
#Mit crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train_cropped),np.array(Y_train_cropped), batch_size=15, epochs=15, validation_data=(np.array(X_validation_cropped), np.array(Y_validation_cropped)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/15
8000/8000 [==============================] - 73s 9ms/step - loss: 0.7556 - mae: 0.0485 - val_loss: 2.0145 - val_mae: 0.1110
Epoch 2/15
8000/8000 [==============================] - 61s 8ms/step - loss: 0.6370 - mae: 0.0419 - val_loss: 0.9144 - val_mae: 0.0594
Epoch 3/15
 795/8000 [=>............................] - ETA: 51s - loss: 0.5913 - mae: 0.0391

KeyboardInterrupt: 

In [4]:
#Mit Dice
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss=bbdc.dice_loss, metrics=['mae'])
history = model.fit(np.array(X_train_cropped),np.array(Y_train_cropped), batch_size=15, epochs=3, validation_data=(np.array(X_validation_cropped), np.array(Y_validation_cropped)))

Train on 8000 samples, validate on 1000 samples
Epoch 1/3
8000/8000 [==============================] - 68s 9ms/step - loss: 0.1913 - mae: 0.0679 - val_loss: 0.3986 - val_mae: 0.1131
Epoch 2/3
8000/8000 [==============================] - 53s 7ms/step - loss: 0.1277 - mae: 0.0443 - val_loss: 0.1378 - val_mae: 0.0463
Epoch 3/3
8000/8000 [==============================] - 58s 7ms/step - loss: 0.1083 - mae: 0.0372 - val_loss: 0.2877 - val_mae: 0.0856


In [ ]:
#Mit binary_crossentropy
model = keras.models.Model(inputs=input_layer, outputs=x)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['mae'])
history = model.fit(np.array(X_train),np.array(Y_train), batch_size=15, epochs=15, validation_data=(np.array(X_validation), np.array(Y_validation)))

In [19]:
prediction = model.predict(np.array(X_test_cropped))
df = bbdc.getPredictionAsSequenceDF(prediction, timepoints, testDataframe)
df.head()

,filename,onset,offset,event_label
0,09001_mix.wav,0.048,1.152,Noise
1,09001_mix.wav,1.200,1.728,Scratching_(performance_technique)
2,09001_mix.wav,1.776,2.400,Shout
3,09001_mix.wav,2.448,3.120,Scratching_(performance_technique)
4,09001_mix.wav,3.168,3.888,Noise
